<h1><center>Introduction</center></h1>

> If you have any confusion in confusion matrix or AUC ROC, this notebook will clear each and every doubt and you will feel more confirtable in dealing with them! 

> The confusion matrix is the base for calculating all the evaluation parametes like accuracy, sensitivity, specificity, precision, recall, f1-score, etc. so its understanding it very important to a data scientist. The AUC ROC is also a important evaluation matrix specially for binary classification but can be applicable to multiclass classification also.

> As the threshold connects the two - confusion matrix and the ROC curve, I decided to explain them together.

> I am using this dataset as it is very popular and doesn't require much preprocessing. So, let's begin!

<h3><center>Importing the libraries</center></h3>

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics

from ipywidgets import interact
import ipywidgets as widgets

import matplotlib.pyplot as plt
import seaborn as sns

sns.set(font = 'Serif', style = 'white', rc = {'axes.facecolor':'#f1f1f1', 'figure.facecolor':'#f1f1f1'})

<h3><center>Reading Data and Preprocessing</center></h3>

> I am not performing any EDA, Data Cleaning, Feature Engineering, etc., as the focus of this notebook is understanding of Confusion Matrix and ROC curve. If you want to see these steps, you can go through [my other notebook](https://www.kaggle.com/sonukiller99/our-drinking-water-is-safe-or-is-it-eda-ml) for the same dataset.

In [ ]:
# Reading the data
df = pd.read_csv('../input/water-potability/water_potability.csv')
df.head()

In [ ]:
# Dropping the missing values
df = df.dropna().reset_index(drop = True)

In [ ]:
# Training data
y = df['Potability']
X = df.iloc[:,:-1]

<h3><center>Creating ML model and Preprocessing</center></h3>

> I choose Random Forest, as it doesn't require any feature scaling or one hot encoding, also it performs pretty good. I have selected some random hyperparamets just to avoid overfitting.

In [ ]:
# Creating the LogisticRegression model
model = RandomForestClassifier(min_samples_leaf = 20, min_samples_split = 30, max_depth = 7).fit(X, y)

<h2><center> Confusion Matrix</center></h2>

> <center><img src = "https://skappal7.files.wordpress.com/2018/08/confusion-matrix.jpg?w=748"></center>

There are few basic terms:

- TP : Positives which we classified correctly
- TN : Negetives which we classified correctly
- FP : Negetives which we classified incorrectly (i.e. classified as positives)
- FN : Positives which we classified incorrectly (i.e. classified as negetives)

#### How the Confusion Matrix is plotted:
> The data goes into the model and it predicts probabilities between 0 and 1. The defalt thresold is set at 0.5, i.e. any probability which is <0.5 is classified as 0 (negetives) and >=0.5 is classified as 1 (positives). 

> This is done for all the input data points whoes acutal lables are known (ground thruth). So now we have actual and predicted 1's (positives) and 0's (negetives) and the confusion matrix is contructed!

#### How to interprete the Confusion Matrix:
> The use of Confusion Matrix varies accross different fields. For eg. in medical, generally the focus is on reducing the False Negetives to minimun (ideally 0), as classifying a patient as no disease who actually has it is very dangerous. So, based on your requirement you can choose which box you want in confusion matrix.

> The boxes in the Confusion Matrix can be combined to provide more information like, sensitivity, specificity, precision, recall, etc. Eg. if you want to know, how many 1's (positives) you have correctly identified then you can choose sensitivity and for 0's (negetives) specificity is used.
> <center> <img src = "https://miro.medium.com/max/576/1*RWwHcEAgsd-yAs7UhRLuVg.png"> </center>

> So, once you understand your need, you can choose among the evaluations parameters.

In [ ]:
def my_confusion_matrix(thresh):

    probability = model.predict_proba(X)
    matrix = pd.DataFrame()

    threshold = thresh
    y_predict = (probability>=threshold).astype(int)[:,1]

    matrix_normal = metrics.confusion_matrix(y, model.predict(X))
    matrix = metrics.confusion_matrix(y, y_predict)

    fig, ((axis1, axis2), (axis3, axis4)) = plt.subplots(2,2, figsize=(20,12), constrained_layout = True)

    # Threshold = 0.5
    axis1 = sns.heatmap(matrix_normal, annot=True, fmt = '.0f', cbar=False, cmap='Blues',
                        linewidths=3, square=True, ax = axis1, annot_kws={"fontsize":30, 'weight':'bold'})
    axis1.set_title(f"Confusion Matrics | Threshold : 0.5", fontsize=16, weight='bold', y=1.05);
    axis1.set_xlabel('Predicted', fontsize=12, weight = 'bold')
    axis1.set_ylabel('Actual', fontsize=12, weight = 'bold')
    axis1.set_xticklabels([0,1], fontsize=12 )
    axis1.set_yticklabels([0,1], fontsize=12, rotation=0)

    # Threshold = given value
    axis3 = sns.heatmap(matrix, annot=True, fmt = '.0f', cbar=False, cmap='Blues',
                        linewidths=3, square=True, ax = axis3, annot_kws={"fontsize":30, 'weight':'bold'})
    axis3.set_title(f"Confusion Matrics | Threshold : {thresh}", fontsize=16, weight='bold', y=1.05, color='green');
    axis3.set_xlabel('Predicted', fontsize=12, weight = 'bold')
    axis3.set_ylabel('Actual', fontsize=12, weight = 'bold')
    axis3.set_xticklabels([0,1], fontsize=12 )
    axis3.set_yticklabels([0,1], fontsize=12, rotation=0);

    # AUC ROC plot
    # calculate the fpr and tpr for all thresholds of the classification
    preds = probability[:,1]
    fpr, tpr, threshold = metrics.roc_curve(y, preds)
    roc_auc = metrics.auc(fpr, tpr)
    
    axis2.set_title('ROC Curve', fontsize=16, weight='bold', y=1.05)
    axis2.plot(fpr, tpr, 'b', label = 'AUC = %0.2f' % roc_auc)
    axis2.legend(loc = 'lower right')
    axis2.plot([0, 1], [0, 1],'r--')
    axis2.set_xlim([0, 1])
    axis2.set_ylim([0, 1])
    axis2.set_ylabel('True Positive Rate', fontsize=14, weight='bold')
    axis2.set_xlabel('False Positive Rate', fontsize=14, weight='bold');
    
    # Plotting tpr and fpr
    fpr, tpr, threshold = metrics.roc_curve(y, y_predict)
    roc_auc = metrics.auc(fpr, tpr)

    axis4.set_title(f'ROC Curve | Threshold : {thresh}', fontsize=16, weight='bold', y=1.05)
    axis4.plot(fpr, tpr, 'b', label = 'AUC = %0.2f' % roc_auc)
    axis4.legend(loc = 'lower right')
    axis4.plot([0, 1], [0, 1],'r--')
    axis4.set_xlim([0, 1])
    axis4.set_ylim([0, 1])
    axis4.set_ylabel('True Positive Rate', fontsize=14, weight='bold')
    axis4.set_xlabel('False Positive Rate', fontsize=14, weight='bold')
    fig.show()

<h3><center>Effect of Threshold on Confusion Matrix</center></h3>

In [ ]:
interact(my_confusion_matrix, thresh=(0,1,0.05));

#### Role of threshold in confusion matrix
> Confusion Matrix is plotted for the default thresold of 0.5, but it can be plotted for other threshold also. As we change the threshold the predicted 1's and 0's will change and thus Confusion Matrix will also change. 

> Note that by changing the threshold, the probabilities predicted by model doesn't change.

> The sum of TP & FN is constant also the sum of TN and FP is constant. So, by changing threshold one parameter increses while other decreses as their sum is constant.

> When thresold is 0, then all the predicted probabilities will become 1 i.e. we have classified all samples with 1's, so we have correctly classified all positives while wrongly classifying all the negetives. Thus, we will have 811 TP, and 1200 FP. 

> You can imagine threshold as a imaginary line which moves from left to right. As we increase the threshold the imaginary line will move from left to right and thus FN will increase and TP will decrease as their sum is constant. Similarly, TN will increse and FP will decrease. When threshold is 1, then we will have 0 FP, and 0 TP.

> Try moving the slider, to get feel the effect of threshold on confusion matrix.

#### This is how confusion matrix works!

<h3><center>AUC ROC Curve</center></h3>

> It is used for identifying how good our model can distinguish between 1's and 0's. First let's understand the True Positive Rate (TPR) and False Positive Rate (FPR). TPR is nothing but sensitivity, i.e. out of total positives how many we have correctly identified and FPR is out of positives that we identified how many were actual positives. This can sound same thing, but if you read it again you will get the difference.

<center><img src="https://bit.ly/3hk9Rzr"></center>

> As the threshold increase (verticle line moves right) the TP and FP decrease so TPR adn FPR also decrease.

> For different values of threshold, the FPR and TPR are calculated and the curve of FPR vs TPR is called as ROC curve. The area under this curve (AUC) tells the performance of the model. More the AOC, better the model can distinguish between postivies and negetives.

- AOC = 0 : Model has classified 1's as 0's and 0's as once, i.e. wrost.
- AOC = 0.5 : Performance is no better than a random chance, i.e. baseline
- AOC = 1 : All 1's are identified as 1 and 0's as 0's perfectly! i.e. best

> When the threshold is 0 or 1, the AUC is 0.5 and for all other values it is >=0.5, meaning the model is performing better than the baseline model as you will see from the plot. 

> The AUC ROC we got for the model is 0.84, it means the model can distinguish between 1's and 0's with 84% confidence.

#### This is concept of AUC ROC!

#### I hope that your doubts have been cleared and you got some understanding of Confusion Matrix and AUC ROC.

<h3><center>If you like it, don't forget to upvote!</center></h3>